## Instalación de PostgreSQL e importación de librerías

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Creación del servidor PostgreSQL
El siguiente código levanta un PostgreSQL en la máquina del google COLAB y lo inicializa, tanto el usuario como la contraseña para acceder a la base de datos se pueden cambiar pero por defecto serán "postgres" ambas.

El siguiente código también genera una base de datos llamada "northwind" que será la que usaremos.

`psql` es el comando del servicio de postgresql, es el que usamos para mandarle comandos a la hora de editar el usuario y crear la base de datos.


In [ ]:
# Importo librería para ejecutar comandos de shell
import subprocess

# Instalamos postgresql server en la máquina del COLAB
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql
!sudo service postgresql start

# Ejecutar el comando para verificar el estado del servicio PostgreSQL
result = subprocess.run(['service', 'postgresql', 'status'], stdout=subprocess.PIPE)
STATUS_OUTPUT = result.stdout.decode('utf-8')
print(STATUS_OUTPUT)

# Verificar si la palabra "online" está presente en la salida del comando y que muestre online.
if "online" in STATUS_OUTPUT:
    print("PostgreSQL se encuentra online :). Se ha levantado la BBDD.")
else:
    print("PostgreSQL no se encuentra online")

# Variables para la configuración (nombre base de datos, usuario, puerto, ...)
SQL_PRACTICE_DATABASE_NAME="northwind"
SQL_PRACTICE_DATABASE_HOST="localhost"
SQL_PRACTICE_DATABASE_PORT=5432
SQL_PRACTICE_DATABASE_USER="postgres"
SQL_PRACTICE_DATABASE_PASS="postgres"

# Dropeamos BBDD porque no le gusta matar sesiones. (Daba error al ejecutar el Collab porque había otra sesión) + Cambio del password para el username `postgres`
!sudo -u postgres psql -U postgres -c "DROP DATABASE {SQL_PRACTICE_DATABASE_NAME} WITH (FORCE);"
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD '{SQL_PRACTICE_DATABASE_PASS}';"

# Creación de la base de datos
!sudo -u postgres psql -U postgres -c "DROP DATABASE IF EXISTS {SQL_PRACTICE_DATABASE_NAME};"
!sudo -u postgres psql -U postgres -c "CREATE DATABASE {SQL_PRACTICE_DATABASE_NAME};"

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
 * Starting PostgreSQL 14 database server
   ...done.
14/main (port 5432): online

PostgreSQL se encuentra online :). Se ha levantado la BBDD.
DROP DATABASE
ALTER ROLE
NOTICE:  database "northwind" does not exist, skipping
DROP DATABASE
CREATE DATABASE


### Importamos las librerías necesarias
- **[psycopg2](https://www.psycopg.org/)**. Librería que nos permite interactuar con servidores y clientes de postgresql
- **pandas**. Librearía conocida para el análisis y transformación de datos.

In [ ]:
import psycopg2
import pandas as pd

## Creación de tablas y carga de datos en el servidor PostgreSQL

Como hemos visto, la [base de datos de Northwind](https://dbdocs.io/akweiwonder3/Northwind-Database?schema=public&view=relationships&table=order_details) tiene varias tablas, el objetivo de esta práctica es familiarizarse con los comandos de consulta de SQL y no de creación de base de datos, por lo que usaremos un script que nos creará todas las tablas y nos cargará todos los datos.

### Creación y configuración base de datos

In [ ]:
# Primero creamos la conexión usando la librearía psycopg2
# https://pypi.org/project/psycopg2/
connection = psycopg2.connect(host=SQL_PRACTICE_DATABASE_HOST,
                              database=SQL_PRACTICE_DATABASE_NAME,
                              user=SQL_PRACTICE_DATABASE_USER,
                              password=SQL_PRACTICE_DATABASE_PASS)
connection.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT);

# Creamos el cursor en la conexión, el cursor es nuestra "puerta de acceso" a la base de datos usando
# la conexión configurada
cursor = connection.cursor()

In [ ]:
# Creación de tablas
%env SQL_PRACTICE_DATABASE_NAME = SQL_PRACTICE_DATABASE_NAME
%env SQL_PRACTICE_DATABASE_HOST = SQL_PRACTICE_DATABASE_HOST
%env SQL_PRACTICE_DATABASE_PORT = SQL_PRACTICE_DATABASE_PORT
%env SQL_PRACTICE_DATABASE_USER = SQL_PRACTICE_DATABASE_USER
%env SQL_PRACTICE_DATABASE_PASS = SQL_PRACTICE_DATABASE_PASS

!PGPASSWORD=$SQL_PRACTICE_DATABASE_PASS psql -q -h $SQL_PRACTICE_DATABASE_HOST -p $SQL_PRACTICE_DATABASE_PORT -U $SQL_PRACTICE_DATABASE_USER -d $SQL_PRACTICE_DATABASE_NAME -f northwind_ddl_full.sql

!PGPASSWORD=$SQL_PRACTICE_DATABASE_PASS psql -q -h $SQL_PRACTICE_DATABASE_HOST -p $SQL_PRACTICE_DATABASE_PORT -U $SQL_PRACTICE_DATABASE_USER -d $SQL_PRACTICE_DATABASE_NAME -f northwind_data.sql

env: SQL_PRACTICE_DATABASE_NAME=SQL_PRACTICE_DATABASE_NAME
env: SQL_PRACTICE_DATABASE_HOST=SQL_PRACTICE_DATABASE_HOST
env: SQL_PRACTICE_DATABASE_PORT=SQL_PRACTICE_DATABASE_PORT
env: SQL_PRACTICE_DATABASE_USER=SQL_PRACTICE_DATABASE_USER
env: SQL_PRACTICE_DATABASE_PASS=SQL_PRACTICE_DATABASE_PASS


# Ejercicios





In [ ]:
# Método de pandas para lanzar consultas SQL y cargarlas en un dataframe
pd.read_sql("SELECT * from categories", connection)

<ipython-input-7-e47007f9d804>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT * from categories", connection)


,category_id,category_name,description,picture
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales",[]
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an...",[]
2,3,Confections,"Desserts, candies, and sweet breads",[]
3,4,Dairy Products,Cheeses,[]
4,5,Grains/Cereals,"Breads, crackers, pasta, and cereal",[]
5,6,Meat/Poultry,Prepared meats,[]
6,7,Produce,Dried fruit and bean curd,[]
7,8,Seafood,Seaweed and fish,[]


In [ ]:
# En lugar de usar cursor.execute(sql) y cursor.fetchall() para mostrar resultados
# mejor usar la función read_sql de Pandas que nos devuelve directamente un dataframe

# Creamos una función más simple para no tener que escribir todo cada vez que lancemos una query
def query(sql):
  return pd.read_sql(sql, connection)

## 1. Informe completo de todos los pedidos (orders) desde el origen de los tiempos
Es decir, sacar todos los Orders tal cual, con toda la información

In [ ]:
# DML(SELECT)
query("SELECT * FROM orders")

<ipython-input-8-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,order_id,customer_id,employee_id,order_date,required_date,shipped_date,ship_via,freight,ship_name,ship_address,ship_city,ship_region,ship_postal_code,ship_country
0,10248,VINET,5,1996-07-04,1996-08-01,1996-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,None,51100,France
1,10249,TOMSP,6,1996-07-05,1996-08-16,1996-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,None,44087,Germany
2,10250,HANAR,4,1996-07-08,1996-08-05,1996-07-12,2,65.83,Hanari Carnes,"Rua do Paço, 67",Rio de Janeiro,RJ,05454-876,Brazil
3,10251,VICTE,3,1996-07-08,1996-08-05,1996-07-15,1,41.34,Victuailles en stock,"2, rue du Commerce",Lyon,None,69004,France
4,10252,SUPRD,4,1996-07-09,1996-08-06,1996-07-11,2,51.30,Suprêmes délices,"Boulevard Tirou, 255",Charleroi,None,B-6000,Belgium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,11073,PERIC,2,1998-05-05,1998-06-02,None,2,24.95,Pericles Comidas clásicas,Calle Dr. Jorge Cash 321,México D.F.,None,05033,Mexico
826,11074,SIMOB,7,1998-05-06,1998-06-03,None,2,18.44,Simons bistro,Vinbæltet 34,Kobenhavn,None,1734,Denmark
827,11075,RICSU,8,1998-05-06,1998-06-03,None,2,6.19,Richter Supermarkt,Starenweg 5,Genève,None,1204,Switzerland
828,11076,BONAP,4,1998-05-06,1998-06-03,None,2,38.28,Bon app',"12, rue des Bouchers",Marseille,None,13008,France


In [ ]:
query("SELECT * FROM customers")

<ipython-input-13-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,customer_id,company_name,contact_name,contact_title,address,city,region,postal_code,country,phone,fax
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,030-0076545
1,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,None,05021,Mexico,(5) 555-4729,(5) 555-3745
2,ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,None,05023,Mexico,(5) 555-3932,None
3,AROUT,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,None,WA1 1DP,UK,(171) 555-7788,(171) 555-6750
4,BERGS,Berglunds snabbköp,Christina Berglund,Order Administrator,Berguvsvägen 8,Luleå,None,S-958 22,Sweden,0921-12 34 65,0921-12 34 67
...,...,...,...,...,...,...,...,...,...,...,...
86,WARTH,Wartian Herkku,Pirkko Koskitalo,Accounting Manager,Torikatu 38,Oulu,None,90110,Finland,981-443655,981-443655
87,WELLI,Wellington Importadora,Paula Parente,Sales Manager,"Rua do Mercado, 12",Resende,SP,08737-363,Brazil,(14) 555-8122,None
88,WHITC,White Clover Markets,Karl Jablonski,Owner,305 - 14th Ave. S. Suite 3B,Seattle,WA,98128,USA,(206) 555-4112,(206) 555-4115
89,WILMK,Wilman Kala,Matti Karttunen,Owner/Marketing Assistant,Keskuskatu 45,Helsinki,None,21240,Finland,90-224 8858,90-224 8858


## 2. Se necesitan dos reports sobre los Customers:
- Lista de Customers/Contacts que son managers
- Lista de Customer/Contacts que no son managers

Ambos reportes tendrán que estar ordenados *alfabéticamente por país* y *nombre* para cargarlos en el CRM (formato específico), además las columnas tienen que tener los siguientes nombres para evitar problemas en la carga: compañia, nombre, posicion, direccion, ciudad, pais.

_Funciones útiles: [SQL Like](https://www.w3schools.com/sql/sql_like.asp), [SQL IN](https://www.w3schools.com/sql/sql_in.asp), [SQL Aliases](https://www.w3schools.com/sql/sql_alias.asp), [DISTINCT](https://www.w3schools.com/sql/sql_distinct.asp)_

In [ ]:
query("SELECT DISTINCT (contact_title) FROM customers")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,contact_title
0,Sales Manager
1,Owner
2,Owner/Marketing Assistant
3,Accounting Manager
4,Sales Associate
5,Sales Agent
6,Assistant Sales Agent
7,Assistant Sales Representative
8,Order Administrator
9,Marketing Manager


In [ ]:
#Lista de Customers/Contacts que son managers
# IN OPERATOR
query("SELECT * FROM customers WHERE contact_title IN ('Sales Manager', 'Accounting Manager', 'Marketing Manager')")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,customer_id,company_name,contact_name,contact_title,address,city,region,postal_code,country,phone,fax
0,BLONP,Blondesddsl père et fils,Frédérique Citeaux,Marketing Manager,"24, place Kléber",Strasbourg,None,67000,France,88.60.15.31,88.60.15.32
1,BOTTM,Bottom-Dollar Markets,Elizabeth Lincoln,Accounting Manager,23 Tsawassen Blvd.,Tsawassen,BC,T2F 8M4,Canada,(604) 555-4729,(604) 555-3745
2,CENTC,Centro comercial Moctezuma,Francisco Chang,Marketing Manager,Sierras de Granada 9993,México D.F.,None,05022,Mexico,(5) 555-3392,(5) 555-7293
3,ERNSH,Ernst Handel,Roland Mendel,Sales Manager,Kirchgasse 6,Graz,None,8010,Austria,7675-3425,7675-3426
4,FISSA,FISSA Fabrica Inter. Salchichas S.A.,Diego Roel,Accounting Manager,"C/ Moralzarzal, 86",Madrid,None,28034,Spain,(91) 555 94 44,(91) 555 55 93
5,FRANK,Frankenversand,Peter Franken,Marketing Manager,Berliner Platz 43,München,None,80805,Germany,089-0877310,089-0877451
6,FRANR,France restauration,Carine Schmitt,Marketing Manager,"54, rue Royale",Nantes,None,44000,France,40.32.21.21,40.32.21.20
7,FURIB,Furia Bacalhau e Frutos do Mar,Lino Rodriguez,Sales Manager,Jardim das rosas n. 32,Lisboa,None,1675,Portugal,(1) 354-2534,(1) 354-2535
8,GALED,Galería del gastrónomo,Eduardo Saavedra,Marketing Manager,"Rambla de Cataluña, 23",Barcelona,None,08022,Spain,(93) 203 4560,(93) 203 4561
9,GODOS,Godos Cocina Típica,José Pedro Freyre,Sales Manager,"C/ Romero, 33",Sevilla,None,41101,Spain,(95) 555 82 82,None


In [ ]:
#Lista de Customer/Contacts que no son managers
# NOT IN OPERATOR
query("SELECT * FROM customers WHERE contact_title NOT IN ('Sales Manager', 'Accounting Manager', 'Marketing Manager')")

<ipython-input-22-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,customer_id,company_name,contact_name,contact_title,address,city,region,postal_code,country,phone,fax
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,030-0076545
1,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,None,05021,Mexico,(5) 555-4729,(5) 555-3745
2,ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,None,05023,Mexico,(5) 555-3932,None
3,AROUT,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,None,WA1 1DP,UK,(171) 555-7788,(171) 555-6750
4,BERGS,Berglunds snabbköp,Christina Berglund,Order Administrator,Berguvsvägen 8,Luleå,None,S-958 22,Sweden,0921-12 34 65,0921-12 34 67
5,BLAUS,Blauer See Delikatessen,Hanna Moos,Sales Representative,Forsterstr. 57,Mannheim,None,68306,Germany,0621-08460,0621-08924
6,BOLID,Bólido Comidas preparadas,Martín Sommer,Owner,"C/ Araquil, 67",Madrid,None,28023,Spain,(91) 555 22 82,(91) 555 91 99
7,BONAP,Bon app',Laurence Lebihan,Owner,"12, rue des Bouchers",Marseille,None,13008,France,91.24.45.40,91.24.45.41
8,BSBEV,B's Beverages,Victoria Ashworth,Sales Representative,Fauntleroy Circus,London,None,EC2 5NT,UK,(171) 555-1212,None
9,CACTU,Cactus Comidas para llevar,Patricio Simpson,Sales Agent,Cerrito 333,Buenos Aires,None,1010,Argentina,(1) 135-5555,(1) 135-4892


In [ ]:
# KNOW THE DIFFERENT JOB TITLES FROM THE EMPLOYEES TABLE
query("SELECT DISTINCT (title) FROM employees")

<ipython-input-22-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,title
0,Inside Sales Coordinator
1,"Vice President, Sales"
2,Sales Manager
3,Sales Representative


In [ ]:
query("SELECT * FROM employees WHERE title IN ('Sales Manager')")

<ipython-input-22-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,employee_id,last_name,first_name,title,title_of_courtesy,birth_date,hire_date,address,city,region,postal_code,country,home_phone,extension,photo,notes,reports_to,photo_path
0,5,Buchanan,Steven,Sales Manager,Mr.,1955-03-04,1993-10-17,14 Garrett Hill,London,None,SW1 8JR,UK,(71) 555-4848,3453,[],Steven Buchanan graduated from St. Andrews Uni...,2,http://accweb/emmployees/buchanan.bmp


## 3. Se necesitan 4 reportes distintos:
1. Lista de Orders de 1996 (DATE)
2. Lista de Employees, con su nombre completo en un único campo, su edad y país
3. Número de Orders hechos desde el 15 de Julio de 1997 (COUNT)
4. Media de días que se tarda en enviar un pedido desde que se hace (AVG)

_Funciones útiles: [CONCAT](https://www.postgresqltutorial.com/postgresql-string-functions/postgresql-concat-function/), [DATE_PART](https://www.postgresql.org/docs/8.1/functions-datetime.html), [COUNT](https://www.w3schools.com/sql/sql_count.asp), [AVG](https://www.w3schools.com/sql/sql_avg.asp)_

In [ ]:
# 1. Lista de Orders de 1996 (DATE)
query ("""
SELECT
order_id,
order_date
FROM orders
WHERE EXTRACT(YEAR FROM order_date) = 1996
""")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,order_id,order_date
0,10248,1996-07-04
1,10249,1996-07-05
2,10250,1996-07-08
3,10251,1996-07-08
4,10252,1996-07-09
...,...,...
147,10395,1996-12-26
148,10396,1996-12-27
149,10397,1996-12-27
150,10398,1996-12-30


In [ ]:
# 1. LISTA DE ORDERS DE 1996 -> YEAR
query("SELECT orders FROM orders WHERE EXTRACT (YEAR FROM order_date) = 1996")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,orders
0,"(10248,VINET,5,1996-07-04,1996-08-01,1996-07-1..."
1,"(10249,TOMSP,6,1996-07-05,1996-08-16,1996-07-1..."
2,"(10250,HANAR,4,1996-07-08,1996-08-05,1996-07-1..."
3,"(10251,VICTE,3,1996-07-08,1996-08-05,1996-07-1..."
4,"(10252,SUPRD,4,1996-07-09,1996-08-06,1996-07-1..."
...,...
147,"(10395,HILAA,6,1996-12-26,1997-01-23,1997-01-0..."
148,"(10396,FRANK,1,1996-12-27,1997-01-10,1997-01-0..."
149,"(10397,PRINI,5,1996-12-27,1997-01-24,1997-01-0..."
150,"(10398,SAVEA,2,1996-12-30,1997-01-27,1997-01-0..."


In [ ]:
# 2. LISTA DE EMPLOYEES CON SU NOMBRE COMPLETO EN UN UNICO CAMPO, SU EDAD Y PAIS -> CONCAT
query("""
SELECT first_name || ' ' || last_name AS name,
(CURRENT_DATE - birth_date)/365 AS age,
country
FROM employees
""")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,name,age,country
0,Andrew Fuller,72,USA
1,Janet Leverling,61,USA
2,Margaret Peacock,87,USA
3,Steven Buchanan,69,UK
4,Michael Suyama,61,UK
5,Robert King,64,UK
6,Laura Callahan,66,USA
7,Anne Dodsworth,58,UK
8,Nancy Davolio,75,USA


In [ ]:
# 3. NUMERO DE ORDERS DESDE 15 JULIO 1997 -> COUNT
query("""
SELECT COUNT(*)
FROM orders
WHERE order_date >= '1997-07-15'
""")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,count
0,479


In [ ]:
# 4. MEDIA DE DIAS QUE SE TARDA EN ENVIAR UN PEDIDO DESDE QUE SE HACE -> AVG (AGGREGATE)
query("SELECT AVG(shipped_date - order_date) AS avg_days FROM orders GROUP BY ship_via")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,avg_days
0,8.571429
1,7.473896
2,9.234921


### EXAMPLES:


In [ ]:
query("SELECT contact_title, COUNT(1) FROM customers WHERE contact_title IN ('Sales Representative', 'Owner') GROUP BY contact_title")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,contact_title,count
0,Owner,17
1,Sales Representative,17


In [ ]:
query("SELECT contact_title, COUNT(1) AS numempleados FROM customers WHERE contact_title IN ('Sales Representative', 'Owner') GROUP BY contact_title")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,contact_title,numempleados
0,Owner,17
1,Sales Representative,17


In [ ]:
query("SELECT contact_name,  contact_title, city FROM customers WHERE contact_title IN ('Sales Representative', 'Owner') GROUP BY contact_name, contact_title, city")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,contact_name,contact_title,city
0,Antonio Moreno,Owner,México D.F.
1,Maria Larsson,Owner,Bräcke
2,Carlos Hernández,Sales Representative,San Cristóbal
3,Guillermo Fernández,Sales Representative,México D.F.
4,Hanna Moos,Sales Representative,Mannheim
5,Miguel Angel Paolino,Owner,México D.F.
6,Henriette Pfalzheim,Owner,Köln
7,Rita Müller,Sales Representative,Stuttgart
8,Jose Pavarotti,Sales Representative,Boise
9,Sergio Gutiérrez,Sales Representative,Buenos Aires


## 4. Reportes para preparar la temporada de Navidad
1. Lista todos los pedidos, saca un campo extra _is_christmas_season_ que sea True si el pedido se hizo en temporada de navidad o False si se hizo fuera de temporada
2. Necesitamos hacer un Sanity Check en la base de datos para comprobar que está todo en orden antes de empezar la temporada de Navidad... ¿Podrías comprobar que el número de pedidos en la tabla **orders** coincide con el número de pedidos de la tabla **order_details**?

3. Queremos enviar un regalo a empleados y clientes. Extrae en un mismo reporte (query) la lista de ambos, los campos que se necesitan son el nombre completo, la dirección, el codigo postal, ciudad y país

Consideramos temporada de Navidad desde el 1 de Diciembre al 31 de Diciembre de cualquier año.

_Funciones útiles: [CASE WHEN](https://www.w3schools.com/sql/sql_case.asp), [DATE_PART](https://www.postgresql.org/docs/8.1/functions-datetime.html), [UNION](https://www.w3schools.com/sql/sql_union.asp), [DISTINCT](https://www.w3schools.com/sql/sql_distinct.asp)_

In [ ]:
# 1- Lista todos los pedidos, saca un campo extra is_christmas_season:
# True si el pedido se hizo en temporada de navidad
# False si se hizo fuera de temporada

#query("""
#SELECT
#DISCTINCT order_date BETWEEN '1996-12-01' AND '1996-12-31' AS is_christmas_season
#FROM orders
#""")

#query("""
#SELECT
#order_date
#EXTRACT(month from order_date) = 12 AS is_christmas_season
#FROM orders
#""")

query ("""
SELECT
*,
CASE
WHEN EXTRACT(month from order_date) = 12 THEN TRUE
ELSE FALSE
END AS is_christmas_season
FROM orders
""")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,order_id,customer_id,employee_id,order_date,required_date,shipped_date,ship_via,freight,ship_name,ship_address,ship_city,ship_region,ship_postal_code,ship_country,is_christmas_season
0,10248,VINET,5,1996-07-04,1996-08-01,1996-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,None,51100,France,False
1,10249,TOMSP,6,1996-07-05,1996-08-16,1996-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,None,44087,Germany,False
2,10250,HANAR,4,1996-07-08,1996-08-05,1996-07-12,2,65.83,Hanari Carnes,"Rua do Paço, 67",Rio de Janeiro,RJ,05454-876,Brazil,False
3,10251,VICTE,3,1996-07-08,1996-08-05,1996-07-15,1,41.34,Victuailles en stock,"2, rue du Commerce",Lyon,None,69004,France,False
4,10252,SUPRD,4,1996-07-09,1996-08-06,1996-07-11,2,51.30,Suprêmes délices,"Boulevard Tirou, 255",Charleroi,None,B-6000,Belgium,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,11073,PERIC,2,1998-05-05,1998-06-02,None,2,24.95,Pericles Comidas clásicas,Calle Dr. Jorge Cash 321,México D.F.,None,05033,Mexico,False
826,11074,SIMOB,7,1998-05-06,1998-06-03,None,2,18.44,Simons bistro,Vinbæltet 34,Kobenhavn,None,1734,Denmark,False
827,11075,RICSU,8,1998-05-06,1998-06-03,None,2,6.19,Richter Supermarkt,Starenweg 5,Genève,None,1204,Switzerland,False
828,11076,BONAP,4,1998-05-06,1998-06-03,None,2,38.28,Bon app',"12, rue des Bouchers",Marseille,None,13008,France,False


In [ ]:
# 2. check order_details
query("""
SELECT
*
FROM order_details
LIMIT 100
""")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,order_id,product_id,unit_price,quantity,discount
0,10248,11,14.0,12,0.00
1,10248,42,9.8,10,0.00
2,10248,72,34.8,5,0.00
3,10249,14,18.6,9,0.00
4,10249,51,42.4,40,0.00
...,...,...,...,...,...
95,10284,27,35.1,15,0.25
96,10284,44,15.5,21,0.00
97,10284,60,27.2,20,0.25
98,10284,67,11.2,5,0.25


In [ ]:
# 2- Before xmas:
#¿Podrías comprobar que el número de pedidos en la tabla orders coincide con
# el número de pedidos de la tabla order_details?

# orders = 830
query("""
SELECT
COUNT(DISTINCT order_id)
FROM orders
""")

# order_details = 830
#query("""
#SELECT
#COUNT(DISTINCT order_id)
#FROM order_details
#""")

# ANOTHER WAY USE FULL JOIN:
#query("""
#SELECT
#COUNT(DISTINCT orders.order_id)
#FROM orders
#FULL JOIN order_details
#ON order.order_id = order_details.order_id
#""")

# TODOS LOS ORDER DETAILS QUE HAY
#query("""
#SELECT
#COUNT( DISTINCT orders.order_id)
#COUNT( DISTINCTorder_details.order_id)
#FROM orders
#FULL JOIN order_details
#ON order.order_id = order_details.order_id
#""")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,count
0,830


In [ ]:
# 3- Send xmas gifts to employees & clients.
# Extract in the same query a list with both, so like full name (first_name + last_name, address, zip code, city, & country
# logical AKA CONCAT || ' ' ||
# UNION MUST HAVE THE SAME NUMBER OF COLUMNS
query("""
SELECT
first_name || ' ' || last_name AS full_name,
address,
postal_code,
city,
country,
'employee' as type
FROM employees
UNION
SELECT
contact_name AS full_name,
address,
postal_code,
city,
country,
'customers' as type
FROM customers
""")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,full_name,address,postal_code,city,country,type
0,Martín Sommer,"C/ Araquil, 67",28023,Madrid,Spain,customers
1,Mario Pontes,"Rua do Paço, 67",05454-876,Rio de Janeiro,Brazil,customers
2,Liz Nixon,89 Jefferson Way Suite 2,97201,Portland,USA,customers
3,Paul Henriot,59 rue de l'Abbaye,51100,Reims,France,customers
4,Rita Müller,Adenauerallee 900,70563,Stuttgart,Germany,customers
...,...,...,...,...,...,...
95,Mary Saveley,"2, rue du Commerce",69004,Lyon,France,customers
96,Sven Ottlieb,Walserweg 21,52066,Aachen,Germany,customers
97,Dominique Perrier,"25, rue Lauriston",75016,Paris,France,customers
98,Maria Larsson,Åkergatan 24,S-844 67,Bräcke,Sweden,customers


## 5. Investigar el retraso de algunos Orders 1/2
Podemos considerar un Order como retrasado cuando la fecha de envío es superior a la fecha requerida.
1. ¿Cuántos Orders hay con retraso?
2. Sacar una lista de todos los orders con el nombre de los shippers (distribuidores)
3. Sacar una lista única de los distribuidores que tienen retrasos
4. Sacar una lista única de todos los distribuidores, marcando los que han tenido retraso alguna vez y los que no, sólo para pedidos hechos durante el año 1998
5. Listar la lista de pedidos con sus empleados asociados que se enviarón después de la fecha requerida (required_date)

_Funciones útiles: [JOINS](https://www.w3schools.com/sql/sql_join.asp), [UNION](https://www.w3schools.com/sql/sql_union.asp), [DISTINCT](https://www.w3schools.com/sql/sql_distinct.asp), [WITH](https://www.postgresql.org/docs/current/queries-with.html#QUERIES-WITH-SELECT)_

In [ ]:
# 1. ¿Cuántos Orders hay con retraso? 37 orders are late for shipping
query("""
SELECT
COUNT(*)
FROM orders
WHERE shipped_date > required_date
""")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,count
0,37


In [ ]:
# 2. Sacar una lista de todos los orders con el nombre de los shippers (distribuidores)
query("""
SELECT
orders.order_id,
shippers.company_name
FROM orders
INNER JOIN shippers
ON orders.ship_via = shippers.shipper_id
""")

# or
query("""
SELECT
o.order_id,
o.ship_name,
s.company_name
FROM orders o
JOIN shippers s ON o.ship_via = s.shipper_id
""")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,order_id,ship_name,company_name
0,10248,Vins et alcools Chevalier,Federal Shipping
1,10249,Toms Spezialitäten,Speedy Express
2,10250,Hanari Carnes,United Package
3,10251,Victuailles en stock,Speedy Express
4,10252,Suprêmes délices,United Package
...,...,...,...
825,11073,Pericles Comidas clásicas,United Package
826,11074,Simons bistro,United Package
827,11075,Richter Supermarkt,United Package
828,11076,Bon app',United Package


In [ ]:
# EXAMPLE JOIN WITH GROUP BY:
#STRING_AGG TO CONCATENATE
query("""
SELECT
s.company_name,
o.ship_name,
COUNT(o.order_id),
STRING_AGG(o.order_id::varchar, ',')
FROM orders o
JOIN shippers s ON o.ship_via = s.shipper_id
GROUP BY s.company_name, o.ship_name
ORDER BY company_name
""")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,company_name,ship_name,count,string_agg
0,Federal Shipping,Alfred's Futterkiste,1,10835
1,Federal Shipping,Ana Trujillo Emparedados y helados,3,"10759,10308,10926"
2,Federal Shipping,Antonio Moreno Taquería,2,"10573,10677"
3,Federal Shipping,Around the Horn,4,"10741,10383,10793,10707"
4,Federal Shipping,Berglunds snabbköp,4,"10733,10444,10837,10384"
...,...,...,...,...
235,United Package,Vins et alcools Chevalier,2,"10737,10295"
236,United Package,Wartian Herkku,5,"10583,10553,10526,10455,10412"
237,United Package,Wellington Importadora,4,"10644,10256,10900,10905"
238,United Package,White Clover Markets,5,"10740,10344,10483,10861,11066"


In [ ]:
# 3. Sacar una lista única de los distribuidores que tienen retrasos
query("""
SELECT
DISTINCT
s.company_name
FROM orders o
LEFT JOIN shippers s ON s.shipper_id = o.ship_via
WHERE shipped_date > required_date
GROUP BY s.company_name
""")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,company_name
0,United Package
1,Speedy Express
2,Federal Shipping


In [ ]:
# 4. Sacar una lista única de todos los distribuidores, marcando los que han tenido retraso alguna vez y los que no, sólo para pedidos hechos durante el año 1998
query ("""
SELECT DISTINCT s.company_name,
CASE WHEN shipped_date > required_date THEN TRUE ELSE FALSE END AS is_late
FROM orders o JOIN shippers s ON s.shipper_id = o.ship_via
WHERE EXTRACT(YEAR FROM order_date) = 1998""")


<ipython-input-8-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,company_name,is_late
0,Federal Shipping,False
1,Federal Shipping,True
2,Speedy Express,False
3,Speedy Express,True
4,United Package,False
5,United Package,True


In [ ]:
# 5. Listar la lista de pedidos con sus empleados asociados que se enviarón después de la fecha requerida (required_date)
query ("""SELECT o.order_id, o.order_date, o.required_date,
o.shipped_date, e.first_name, e.last_name
FROM orders o JOIN employees e ON e.employee_id = o.employee_id
WHERE shipped_date > required_date """)

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,order_id,order_date,required_date,shipped_date,first_name,last_name
0,10264,1996-07-24,1996-08-21,1996-08-23,Michael,Suyama
1,10271,1996-08-01,1996-08-29,1996-08-30,Michael,Suyama
2,10280,1996-08-14,1996-09-11,1996-09-12,Andrew,Fuller
3,10302,1996-09-10,1996-10-08,1996-10-09,Margaret,Peacock
4,10309,1996-09-19,1996-10-17,1996-10-23,Janet,Leverling
5,10320,1996-10-03,1996-10-17,1996-10-18,Steven,Buchanan
6,10380,1996-12-12,1997-01-09,1997-01-16,Laura,Callahan
7,10423,1997-01-23,1997-02-06,1997-02-24,Michael,Suyama
8,10427,1997-01-27,1997-02-24,1997-03-03,Margaret,Peacock
9,10433,1997-02-03,1997-03-03,1997-03-04,Janet,Leverling


## 6. Investigar el retraso de algunos orders 2/2
1. ¿Cuántos pedidos retrasados hay por año?
2. ¿Cuántos pedidos retrasados tiene cada distribuidor? ¿Y por año?
3. Top 2 empleados con más pedidos retrasados
4. Top 2 meses con más pedidos retrasados (mes y año)
5. ¿Hay alguna relación entre el número de productos de un pedido y el retraso?

_Funciones útiles: [GROUP BY](https://www.w3schools.com/sql/sql_groupby.asp), [JOINS](https://www.w3schools.com/sql/sql_join.asp)_

In [ ]:
#1.Cuántos pedidos retrasados hay por año?
# Using EXTRACT instead of YEAR
query("SELECT EXTRACT(YEAR FROM orders.shipped_date) AS year, COUNT(*) AS late_orders FROM orders WHERE orders.shipped_date > orders.required_date GROUP BY EXTRACT(YEAR FROM orders.shipped_date)")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,year,late_orders
0,1998.0,11
1,1996.0,6
2,1997.0,20


In [ ]:
# 2. ¿Cuántos pedidos retrasados tiene cada distribuidor?
# AGREGGATE FUNCTION: COUNT()
query("SELECT shippers.company_name, COUNT (orders.order_id) AS late_orders FROM shippers JOIN orders ON shippers.shipper_id=orders.ship_via WHERE orders.shipped_date IS NOT NULL AND orders.shipped_date>orders.required_date GROUP BY shippers.company_name")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,company_name,late_orders
0,United Package,16
1,Speedy Express,12
2,Federal Shipping,9


In [ ]:
# 2. ¿Y por año?
# Using EXTRACT instead of YEAR to get the year from the order date
# AGREGGATE FUNCTION:  COUNT()
query ("SELECT EXTRACT(YEAR FROM orders.order_date) AS year, shippers.company_name, COUNT (orders.order_id) AS late_orders FROM shippers JOIN orders ON shippers.shipper_id=orders.ship_via WHERE orders.shipped_date IS NOT NULL AND orders.shipped_date>orders.required_date GROUP BY EXTRACT(YEAR FROM orders.order_date), shippers.company_name ORDER BY , shippers.company_name ")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,year,company_name,late_orders
0,1996.0,Federal Shipping,3
1,1996.0,Speedy Express,2
2,1996.0,United Package,2
3,1997.0,Federal Shipping,5
4,1997.0,Speedy Express,6
5,1997.0,United Package,11
6,1998.0,Federal Shipping,1
7,1998.0,Speedy Express,4
8,1998.0,United Package,3


In [ ]:
# 3. Top 2 empleados con más pedidos retrasados employees table & orders tables
# AGREGGATE FUNCTION: COUNT()
query("""
SELECT employees.employee_id, employees.first_name || ' ' || employees.last_name AS full_name, COUNT(*) AS late_orders
FROM orders
JOIN employees ON orders.employee_id = employees.employee_id
WHERE required_date > shipped_date
GROUP BY employees.employee_id, full_name
ORDER BY late_orders DESC
LIMIT 2
""")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,employee_id,full_name,late_orders
0,4,Margaret Peacock,141
1,3,Janet Leverling,122


In [ ]:
# 4. Top 2 meses con más pedidos retrasados (mes y año)
# EXTRACT function to extract year and month.
# AGREGGATE FUNCTION: COUNT()
query("SELECT EXTRACT(YEAR FROM orders.order_date) AS year, EXTRACT(MONTH FROM orders.order_date) AS month, COUNT(orders.order_id) AS late_orders FROM shippers JOIN orders ON shippers.shipper_id = orders.ship_via WHERE orders.shipped_date IS NOT NULL AND orders.shipped_date > orders.required_date GROUP BY EXTRACT(YEAR FROM orders.order_date), EXTRACT(MONTH FROM orders.order_date) ORDER BY late_orders DESC LIMIT 2")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,year,month,late_orders
0,1998.0,3.0,4
1,1998.0,1.0,4


In [ ]:
# 5. ¿Hay alguna relación entre el número de productos de un pedido y el retraso? orders table & orders details table
# AGREGGATE FUNCTION: COUNT()
query("SELECT COUNT(order_details.product_id) AS number_of_products, CASE WHEN orders.shipped_date > orders.required_date THEN orders.shipped_date - orders.required_date ELSE 0 END AS late_orders FROM orders JOIN order_details ON orders.order_id = order_details.order_id WHERE orders.shipped_date IS NOT NULL GROUP BY orders.order_id")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,number_of_products,late_orders
0,3,0
1,1,0
2,3,0
3,2,18
4,3,0
...,...,...
804,3,0
805,2,0
806,1,0
807,4,0


## 7. Estudio de los empleados
1. ¿Cuál es la distribución de empleados en los distintos territorios?
2. ¿Cuál es la distribución de los empleados en estos rangos de edad 30-40, 40-60, 60+?
3. Reporte que muestre el número de empleados y clientes que hay en cada ciudad que tenga empleados (es decir, no estamos interesados en las ciudades sin empleados)

_Funciones útiles: [CASE WHEN](https://www.w3schools.com/sql/sql_case.asp), [DATE_PART](https://www.postgresql.org/docs/8.1/functions-datetime.html), [UNION](https://www.w3schools.com/sql/sql_union.asp)_

In [ ]:
# 1. ¿Cuál es la distribución de empleados en los distintos territorios?
# AGREGGATE FUNCTION: COUNT()
query("SELECT t.territory_description, r.region_description AS cardinal_directions, COUNT(e.employee_id) AS number_of_employees FROM territories t LEFT JOIN employee_territories et ON t.territory_id = et.territory_id LEFT JOIN employees e ON et.employee_id = e.employee_id LEFt JOIN region r ON t.region_id = r.region_id GROUP BY t.territory_description, r.region_description")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,territory_description,cardinal_directions,number_of_employees
0,Phoenix,Western,1
1,San Francisco,Western,1
2,Wilton,Eastern,1
3,Mellvile,Eastern,1
4,Seattle,Western,1
5,Dallas,Southern,0
6,Roseville,Northern,1
7,Cary,Eastern,1
8,Hoffman Estates,Western,1
9,Louisville,Eastern,1


In [ ]:
# 2.  ¿Cuál es la distribución de los empleados en estos rangos de edad 30-40, 40-60, 60+?
query("SELECT CASE WHEN (CURRENT_DATE - birth_date) / 365 BETWEEN 30 AND 40 THEN '30-40' WHEN (CURRENT_DATE - birth_date) / 365 BETWEEN 41 AND 60 THEN '40-60' WHEN (CURRENT_DATE - birth_date) / 365 > 60 THEN '60+' ELSE 'Less than 30' END AS age_range, COUNT(*) AS number_of_employees FROM employees WHERE birth_date IS NOT NULL  GROUP BY age_range")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,age_range,number_of_employees
0,40-60,1
1,60+,8


In [ ]:
# 2. OR USING DATEPART
query("SELECT CASE WHEN DATE_PART('year', CURRENT_DATE) - DATE_PART('year', birth_date) BETWEEN 30 AND 40 THEN '30-40' WHEN DATE_PART('year', CURRENT_DATE) - DATE_PART('year', birth_date) BETWEEN 41 AND 60 THEN '40-60' WHEN DATE_PART('year', CURRENT_DATE) - DATE_PART('year', birth_date) > 60 THEN '60+' ELSE 'Less than 30' END AS age_range, COUNT(*) AS number_of_employees FROM employees WHERE birth_date IS NOT NULL GROUP BY age_range")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,age_range,number_of_employees
0,40-60,1
1,60+,8


In [ ]:
# 3. Reporte que muestre el número de empleados y clientes que hay en cada ciudad que tenga empleados  (es decir, no estamos interesados en las ciudades sin empleados)
 # AGREGGATE FUNCTION: COUNT()
query("SELECT e.city AS City, COUNT(DISTINCT e.employee_id) AS number_of_employees, COUNT(DISTINCT c.customer_id) AS customer_count FROM employees e LEFT JOIN employee_territories et ON e.employee_id=et.employee_id LEFT JOIN territories t ON et.territory_id=t.territory_id LEFT JOIN customers c ON c.city=e.city WHERE e.city IS NOT NULL GROUP BY e.city HAVING COUNT(DISTINCT e.employee_id)>0")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,city,number_of_employees,customer_count
0,Kirkland,1,1
1,London,4,6
2,Redmond,1,0
3,Seattle,2,1
4,Tacoma,1,0


## 8. Eficiencia de la operación según la relación empleado - cliente - pedido
1. Teniendo en cuenta los pedidos gestionados por cada empleado, ¿cuántos clientes gestiona de media cada empleado?
2. ¿Cuál es el empleado (nombre) que ha gestionado más clientes? ¿Y más pedidos?
3. Saca una lista con empleados (nombre) que incluya sólo aquellos que han gestionado más de 100 pedidos


In [ ]:
# 1. Teniendo en cuenta los pedidos gestionados por cada empleado, ¿cuántos clientes gestiona de media cada empleado?
#subquery -> to know the number of clientes being managed by each employee
# AGREGGATE FUNCTION: AVG() & COUNT()
query("SELECT AVG(clients_per_employee)AS avg_clients_per_employee FROM (SELECT e.employee_id, COUNT(DISTINCT c.customer_id) AS clients_per_employee FROM employees e LEFT JOIN orders o ON e.employee_id=o.employee_id LEFT JOIN customers c ON o.customer_id=c.customer_id GROUP BY e.employee_id) AS subquery")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,avg_clients_per_employee
0,51.555556


In [ ]:
# 2.  ¿Cuál es el empleado (nombre) que ha gestionado más clientes?
# AGREGGATE FUNCTION: COUNT()
query("SELECT e.employee_id, e.first_name || ' ' || e.last_name AS full_name, COUNT(DISTINCT c.customer_id)AS number_of_clients_per_employee FROM employees e LEFT JOIN orders o ON e.employee_id=o.employee_id LEFT JOIN customers c ON o.customer_id=c.customer_id GROUP BY e.employee_id ORDER BY number_of_clients_per_employee DESC LIMIT 1")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,employee_id,full_name,number_of_clients_per_employee
0,4,Margaret Peacock,75


In [ ]:
#2. ¿Y más pedidos?
# AGREGGATE FUNCTION: COUNT()
query("SELECT e.employee_id, e.first_name || ' ' || e.last_name AS full_name, COUNT(o.order_id)AS number_of_orders_per_employee FROM employees e LEFT JOIN orders o ON e.employee_id=o.employee_id GROUP BY e.employee_id ORDER BY number_of_orders_per_employee DESC LIMIT 1")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,employee_id,full_name,number_of_orders_per_employee
0,4,Margaret Peacock,156


In [ ]:
# 3. Saca una lista con empleados (nombre) que incluya sólo aquellos que han gestionado más de 100 pedidos
# HAVING -> to filter those employees that have more than 100 orders
# AGREGGATE FUNCTION: COUNT()
query("SELECT e.employee_id, e.first_name || ' ' || e.last_name AS full_name, COUNT(o.order_id) AS number_of_orders_per_employee FROM employees e LEFT JOIN orders o ON e.employee_id=o.employee_id GROUP BY e.employee_id HAVING COUNT(o.order_id)>100")

<ipython-input-13-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,employee_id,full_name,number_of_orders_per_employee
0,4,Margaret Peacock,156
1,3,Janet Leverling,127
2,1,Nancy Davolio,123
3,8,Laura Callahan,104


## 9. Reporte que incluya un resumen de las siguientes métricas (todo en la misma tabla)
1. Número total de pedidos
2. Número total de pedidos retrasados
3. % de pedidos retrasados
4. Media de pedidos gestionados por empleado

Todas esas métricas deberán de aparecer por año y més (el ejemplo de abajo es orientativo, los números no son reales):

| año     | mes | total_pedidos | pedidos_retrasados | % pedidos retrasados | pedidos_por_empleado
|----------|----------|----------|----------|----------|----------|
| 1996    |  enero  | 134 | 43 |0.03 |19 |
| 1996    | febrero   | 532 | 63 |0.02 |103 |
| 1996    | marzo   | 149 | 34 |0.18 |25 |

In [ ]:
#DATEPART() function
# CONVERT()
query("SELECT DATE_PART('year', order_date) AS year, DATE_PART('month', order_date) AS month, COUNT(*) AS total_number_of_orders, COUNT(CASE WHEN shipped_date > required_date THEN 1 END) AS late_orders, COUNT(CASE WHEN shipped_date > required_date THEN 1 END)::decimal / COUNT(*) * 100 AS percentage_orders_delayed, AVG(COUNT(*)) OVER (PARTITION BY orders.employee_id) AS avg_orders_manage_per_employee FROM orders JOIN employees ON orders.employee_id = employees.employee_id GROUP BY year, month, orders.employee_id ORDER BY year, month")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,year,month,total_number_of_orders,late_orders,percentage_orders_delayed,avg_orders_manage_per_employee
0,1996.0,7.0,2,1,50.0,3.190476
1,1996.0,7.0,1,0,0.0,4.173913
2,1996.0,7.0,2,0,0.0,2.388889
3,1996.0,7.0,1,0,0.0,5.347826
4,1996.0,7.0,3,0,0.0,2.333333
...,...,...,...,...,...,...
187,1998.0,5.0,2,0,0.0,6.782609
188,1998.0,5.0,3,0,0.0,4.521739
189,1998.0,5.0,2,0,0.0,4.173913
190,1998.0,5.0,5,0,0.0,5.347826


In [ ]:
# SAME ANSWER BUT INSTEAD CASTING A FLOAT
query("SELECT DATE_PART('year', order_date) AS year, DATE_PART('month', order_date) AS month, COUNT(*) AS total_number_of_orders, COUNT(CASE WHEN shipped_date > required_date THEN 1 END) AS late_orders, CAST(COUNT(CASE WHEN shipped_date > required_date THEN 1 END) AS FLOAT) / COUNT(*) * 100 AS percentage_orders_delayed,  AVG(COUNT(*)) OVER (PARTITION BY orders.employee_id) AS avg_orders_manage_per_employee FROM orders JOIN employees ON orders.employee_id = employees.employee_id GROUP BY year, month, orders.employee_id ORDER BY year, month")

<ipython-input-8-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,year,month,total_number_of_orders,late_orders,percentage_orders_delayed,avg_orders_manage_per_employee
0,1996.0,7.0,2,1,50.0,3.190476
1,1996.0,7.0,1,0,0.0,4.173913
2,1996.0,7.0,2,0,0.0,2.388889
3,1996.0,7.0,1,0,0.0,5.347826
4,1996.0,7.0,3,0,0.0,2.333333
...,...,...,...,...,...,...
187,1998.0,5.0,2,0,0.0,6.782609
188,1998.0,5.0,3,0,0.0,4.521739
189,1998.0,5.0,2,0,0.0,4.173913
190,1998.0,5.0,5,0,0.0,5.347826


In [ ]:
# NOTE:
# When we don't use CONVERT NOR CAST:
query("SELECT DATE_PART('year', order_date) AS year, DATE_PART('month', order_date) AS month, COUNT(*) AS total_number_of_orders, COUNT(CASE WHEN shipped_date > required_date THEN 1 END) AS late_orders, COUNT(CASE WHEN shipped_date > required_date THEN 1 END) / COUNT(*) * 100 AS percentage_orders_delayed, AVG(COUNT(*)) OVER (PARTITION BY orders.employee_id) AS avg_orders_manage_per_employee FROM orders JOIN employees ON orders.employee_id = employees.employee_id GROUP BY year, month, orders.employee_id ORDER BY year, month")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,year,month,total_number_of_orders,late_orders,percentage_orders_delayed,avg_orders_manage_per_employee
0,1996.0,7.0,2,1,0,3.190476
1,1996.0,7.0,1,0,0,4.173913
2,1996.0,7.0,2,0,0,2.388889
3,1996.0,7.0,1,0,0,5.347826
4,1996.0,7.0,3,0,0,2.333333
...,...,...,...,...,...,...
187,1998.0,5.0,2,0,0,6.782609
188,1998.0,5.0,3,0,0,4.521739
189,1998.0,5.0,2,0,0,4.173913
190,1998.0,5.0,5,0,0,5.347826


## 10. Comprobaciones finales:
1. ¿Cuáles son las categorías que tienen los productos más caros?
2. ¿Cuáles son los productos (nombre) que tienen unidades en orden y cuáles son sus categorías y distribuidores?
3. ¿Cuál es el precio total de cada pedido?

In [ ]:
# 1. ¿Cuáles son las categorías que tienen los productos más caros? categories table & products table
# AGGREGATE FUNCTION: MAX()
query("SELECT c.category_name, MAX(p.unit_price) AS max_unit_price FROM products p JOIN categories c ON p.category_id=c.category_id GROUP BY c.category_name ORDER BY max_unit_price DESC ")

<ipython-input-22-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,category_name,max_unit_price
0,Beverages,263.50
1,Meat/Poultry,123.79
2,Confections,81.00
3,Seafood,62.50
4,Dairy Products,55.00
5,Produce,53.00
6,Condiments,43.90
7,Grains/Cereals,38.00


In [ ]:
# 2. ¿Cuáles son los productos (nombre) que tienen unidades en orden y cuáles son sus categorías y distribuidores? categories table, products table, & suppliers table
query("SELECT p.product_name, c.category_name, s.company_name FROM products p JOIN categories c ON p.category_id=c.category_id JOIN suppliers s ON p.supplier_id=s.supplier_id ORDER BY p.units_in_stock")

<ipython-input-22-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,product_name,category_name,company_name
0,Thüringer Rostbratwurst,Meat/Poultry,Plutzer Lebensmittelgroßmärkte AG
1,Gorgonzola Telino,Dairy Products,Formaggi Fortini s.r.l.
2,Alice Mutton,Meat/Poultry,"Pavlova, Ltd."
3,Chef Anton's Gumbo Mix,Condiments,New Orleans Cajun Delights
4,Perth Pasties,Meat/Poultry,"G'day, Mate"
...,...,...,...
72,Sirop d'érable,Condiments,Forêts d'érables
73,Pâté chinois,Meat/Poultry,Ma Maison
74,Grandma's Boysenberry Spread,Condiments,Grandma Kelly's Homestead
75,Boston Crab Meat,Seafood,New England Seafood Cannery


In [ ]:
# 3. ¿Cuál es el precio total de cada pedido? orders table, products table, & order details table
# AGGREGATE FUNCTION: SUM ()
query("SELECT o.order_id, SUM(od.quantity*p.unit_price) AS total_price_per_order FROM orders o JOIN order_details od ON o.order_id=od.order_id JOIN products p ON od.product_id=p.product_id GROUP BY o.order_id")

<ipython-input-22-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,order_id,total_price_per_order
0,11038,750.999998
1,10782,12.500000
2,10725,287.799995
3,10423,1275.000000
4,10518,4150.050007
...,...,...
825,10707,1704.000000
826,10826,730.000000
827,10371,114.000000
828,10575,2147.399986


In [ ]:
# USING ROUND() TO ROUND
# AGGREGATE FUNCTION: SUM ()
query("SELECT o.order_id, ROUND(SUM(od.quantity*p.unit_price)) AS total_price_per_order FROM orders o JOIN order_details od ON o.order_id=od.order_id JOIN products p ON od.product_id=p.product_id GROUP BY o.order_id")

<ipython-input-22-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,order_id,total_price_per_order
0,11038,751.0
1,10782,12.0
2,10725,288.0
3,10423,1275.0
4,10518,4150.0
...,...,...
825,10707,1704.0
826,10826,730.0
827,10371,114.0
828,10575,2147.0


In [ ]:
# USING CEIL() FUNCTION TO GET THE SMALLEST INT THAT IS >= TO THAT INT
# AGGREGATE FUNCTION: SUM ()
query("SELECT o.order_id, CEIL(SUM(od.quantity*p.unit_price)) AS total_price_per_order FROM orders o JOIN order_details od ON o.order_id=od.order_id JOIN products p ON od.product_id=p.product_id GROUP BY o.order_id")

<ipython-input-22-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,order_id,total_price_per_order
0,11038,751.0
1,10782,13.0
2,10725,288.0
3,10423,1275.0
4,10518,4151.0
...,...,...
825,10707,1704.0
826,10826,730.0
827,10371,114.0
828,10575,2148.0


## Extra: Window Functions
Aunque esta sección no se evaluará, se recomienda la realización por parte del alumno que quiera profundizar y mejorar su nivel de SQL.
1. Por cada pedido y producto de Order Details, crear una columna llamada __order_page__ indicando el número de página de cada pedido (es decir, un mismo pedido con 3 productos tendrá un producto con un order page 1, otro con un order page 2 y otro con un order page 3)
2. Sacar la evolución del beneficio total obtenido por los pedidos a lo largo del tiempo (beneficio acumulado)
3. Asignar a cada empleado un ranking numérico según el beneficio generado por la gestión de pedidos, es decir el empleado con la posicion 1 será el que haya tenido más beneficio en ventas
4. Reporte donde se muestre el ID del pedido, el empleado que lo gestionó, la fecha, y el acumulado de pedidos de ese empleado hasta ese momento
5. Crear un reporte donde se muestre la evolución del beneficio por mes y año, además queremos que cada año el acumulado sea independiente, es decir que el 1 de Enero el acumulado se reinicie a 0 (la tabla tendrá tres columnas: año, mes y beneficio)

In [ ]:
# 1. Por cada pedido y producto de Order Details, crear una columna llamada order_page indicando el número de página de cada pedido (es decir, un mismo pedido con 3 productos tendrá un producto con un order page 1, otro con un order page 2 y otro con un order page 3)
# CEIL() FUNCTION
# COUNT() FUNCTION
# WINDOWS FUNCTION:
# OVER CLAUSE
# PARTITION BY -> used with window functions for partitioning data
query("SELECT od.order_id, od.product_id, od.quantity, CEIL(COUNT(od.product_id)OVER(PARTITION BY od.order_id)/10.0) AS order_page FROM order_details od")

<ipython-input-22-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,order_id,product_id,quantity,order_page
0,10248,11,12,1.0
1,10248,42,10,1.0
2,10248,72,5,1.0
3,10249,14,9,1.0
4,10249,51,40,1.0
...,...,...,...,...
2150,11077,64,2,3.0
2151,11077,66,1,3.0
2152,11077,73,2,3.0
2153,11077,75,4,3.0


#### Notes
`Window functions` apply to `aggregate` and `ranking` functions over a particular window (set of rows). OVER clause is used with window functions to define that window. OVER clause does two things :

`Partitions` rows to form a set of rows. (PARTITION BY clause is used)
Orders rows within those partitions into a particular order. (ORDER BY clause is used)

**Window functions can be called in the SELECT statement or in the ORDER BY clause. However, they can never be called in the WHERE clause.**

Value window functions:
*   FIRST_VALUE()
*   LAG()
*   LAST_VALUE()
*   LEAD()

Ranking window functions:
*   CUME_DIST()
*   DENSE_RANK()
*   NTILE()
*   PERCENT_RANK()
*   RANK()
*   ROW_NUMBER()

`margen de beneficio`
MB=(P-C)/P

`beneficio total`
Beneficio Total = Ingresos Totales - Costos Totales


In [ ]:
# 2. Sacar la evolución del beneficio total obtenido por los pedidos a lo largo del tiempo (beneficio acumulado) order details table & product table
# DATE_TRUNC()
# total_benefit = total_income - total_costs

In [ ]:
# 3. Asignar a cada empleado un ranking numérico según el beneficio generado por la gestión de pedidos, es decir el empleado con la posicion 1 será el que haya tenido más beneficio en ventas orders table & employees table
# GOUP BY employees
# employees tables DESC
#orders table

In [ ]:
# 4. Reporte donde se muestre el ID del pedido, el empleado que lo gestionó, la fecha, y el acumulado de pedidos de ese empleado hasta ese momento
# CONCAT 1st name + last name
# COUNT() FUNCTION
# WINDOWS FUNCTION:
# OVER CLAUSE
# PARTITION BY -> used with window functions for partitioning data
query("SELECT o.order_id, e.first_name || '' || e.last_name AS full_name, o.order_date, COUNT(o.order_id)OVER (PARTITION BY o.employee_id ORDER BY o.order_date)AS accumulated_orders FROM orders o JOIN employees e ON o.employee_id=e.employee_id ORDER BY o.order_date")

<ipython-input-7-68c59852a541>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,order_id,full_name,order_date,accumulated_orders
0,10248,StevenBuchanan,1996-07-04,1
1,10249,MichaelSuyama,1996-07-05,1
2,10250,MargaretPeacock,1996-07-08,1
3,10251,JanetLeverling,1996-07-08,1
4,10252,MargaretPeacock,1996-07-09,2
...,...,...,...,...
825,11072,MargaretPeacock,1998-05-05,155
826,11077,NancyDavolio,1998-05-06,123
827,11074,RobertKing,1998-05-06,72
828,11076,MargaretPeacock,1998-05-06,156


In [ ]:
# 5. Crear un reporte donde se muestre la evolución del beneficio por mes y año, además queremos que cada año el acumulado sea independiente, es decir que el 1 de Enero el acumulado se reinicie a 0 (la tabla tendrá tres columnas: año, mes y beneficio)